# Crossvalidation and hyperparameter selection  

In [2]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [3]:
df_X, temp = load_diabetes(return_X_y=True, as_frame=True)
s_y = temp.squeeze()
s_y

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [4]:
def leastSquares(df_X, s_y):
    
    # Insert the column of ones
    df_X_1 = df_X.copy()
    df_X_1.insert(0, '', 1)

    # Get the y vector
    y_data = s_y.to_numpy()

    # One-liner to calculate beta_hat
    beta_hat = np.linalg.inv(df_X_1.transpose() @ df_X_1) @ df_X_1.transpose() @ y_data
    
    return beta_hat
leastSquares(df_X, s_y)

0     152.133484
1     -10.012198
2    -239.819089
3     519.839787
4     324.390428
5    -792.184162
6     476.745838
7     101.044570
8     177.064176
9     751.279321
10     67.625386
dtype: float64

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [5]:
def getDataFolds(df_X, s_y, k):
    
    # make list size lenght(df_X) with random numbers between 1 and k
    random_assignments = np.random.randint(1, k + 1, size=df_X.shape[0])
    
    # add this list as a column to both dfs
    df_X = df_X.join(pd.DataFrame(random_assignments, columns=['fold_n']))
    df_sy = s_y.to_frame()
    df_sy = df_sy.join(pd.DataFrame(random_assignments, columns=['fold_n']))
    
    # create a dictionary with keys from the fold_n columns, but drop fold_n
    keys = [i for i in range(1, k+1)]
    dict_k_df_X = dict(tuple(df_X.groupby('fold_n')))
    dict_k_s_y = dict(tuple(df_sy.groupby('fold_n')))
    
    # drop the fold number from each dataframe
    dict_k_df_X = dict(zip(keys,[x.drop(columns=['fold_n']) for x in dict_k_df_X.values()])) 
    dict_k_s_y = dict(zip(keys,[x.drop(columns=['fold_n']) for x in dict_k_s_y.values()]))
    
    return dict_k_df_X, dict_k_s_y
        
# sniff
x_dict,y_dict = getDataFolds(df_X, s_y, 5)

s = 0
for i in range(1, 6):
    s += x_dict[i].shape[0]
    print("dict_k_df_X | fold number {} | num rows: {}".format(i, x_dict[i].shape[0]))
print("sum dict_k_df_X = {}".format(s))

print("\n-------------------------\n")

s = 0
for i in range(1, 6):
    s += y_dict[i].shape[0]
    print("dict_k_s_y | fold number {} | num rows: {}".format(i, y_dict[i].shape[0]))
print("sum dict_k_s_y = {}".format(s))


dict_k_df_X | fold number 1 | num rows: 102
dict_k_df_X | fold number 2 | num rows: 83
dict_k_df_X | fold number 3 | num rows: 78
dict_k_df_X | fold number 4 | num rows: 95
dict_k_df_X | fold number 5 | num rows: 84
sum dict_k_df_X = 442

-------------------------

dict_k_s_y | fold number 1 | num rows: 102
dict_k_s_y | fold number 2 | num rows: 83
dict_k_s_y | fold number 3 | num rows: 78
dict_k_s_y | fold number 4 | num rows: 95
dict_k_s_y | fold number 5 | num rows: 84
sum dict_k_s_y = 442


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [6]:
def MAE(s_y, s_y_hat):
    return np.sum(np.abs(s_y - s_y_hat) / s_y.shape[0])


x = np.array([1,2,3])
y = np.array([2,2,3])
MAE(x, y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [9]:
def crossValidate(df_x, s_y, k):
    
    # get the folded data
    dict_k_df_X, dict_k_df_y = getDataFolds(df_x, s_y, k)
    
    # store MAEs
    MAEs = []
    
    # for every fold (k)
    for i in range(1, k+1):
        
        # get the test set
        test_X_df = dict_k_df_X[i]
        test_y_df = dict_k_df_y[i]
        
        # create the train set
        train_X_df = pd.DataFrame()
        train_y_df = pd.DataFrame()
        for j in range(1, k+1):
            
            # skip the test set
            if (j != i):
                train_X_df = train_X_df.append(dict_k_df_X[j])
                train_y_df = train_y_df.append(dict_k_df_y[j])

        # calculate beta-hat
        beta_hat = leastSquares(train_X_df, train_y_df.squeeze())
        
        # calculate s_y_hat predictions
        s_y_hat = beta_hat[0] + test_X_df.to_numpy().dot(beta_hat[1:])

        # calculate MAE
        MAEs.append(MAE(test_y_df.squeeze().to_numpy(), s_y_hat))
    
    print(test_X_df)
    return min(MAEs), max(MAEs), np.mean(MAEs)
        
minMAE, maxMAE, avgMAE = crossValidate(df_X, s_y, 5)

print("Min MAE: {}\nMaX MAE: {} \nAverage MAE: {}\n".format(minMAE, maxMAE, avgMAE))


          age       sex       bmi        bp        s1        s2        s3  \
12   0.016281 -0.044642 -0.028840 -0.009113 -0.004321 -0.009769  0.044958   
13   0.005383  0.050680 -0.001895  0.008101 -0.004321 -0.015719 -0.002903   
17   0.070769  0.050680  0.012117  0.056301  0.034206  0.049416 -0.039719   
29   0.067136  0.050680 -0.006206  0.063187 -0.042848 -0.095885  0.052322   
34   0.016281 -0.044642 -0.063330 -0.057314 -0.057983 -0.048912  0.008142   
..        ...       ...       ...       ...       ...       ...       ...   
417 -0.052738 -0.044642  0.071397 -0.074528 -0.015328 -0.001314  0.004460   
419 -0.020045 -0.044642 -0.054707 -0.053871 -0.066239 -0.057367  0.011824   
423  0.009016  0.050680 -0.039618  0.028758  0.038334  0.073529 -0.072854   
426  0.030811  0.050680 -0.034229  0.043677  0.057597  0.068831 -0.032356   
429 -0.041840 -0.044642 -0.033151 -0.022885  0.046589  0.041587  0.056003   

           s4        s5        s6  
12  -0.039493 -0.030751 -0.042499  
13 

# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [ ]:
df_X, temp = load_iris(return_X_y=True, as_frame=True)
s_y = temp.squeeze()
s_y.value_counts()

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [ ]:
dict_df_X, dict_s_y = getDataFolds(df_X, s_y, 5)
s = 0
for i in range(1, 6):
    s += dict_df_X[i].shape[0]
    print("dict_df_X | fold number {} | num rows: {}".format(i, dict_df_X[i].shape[0]))
print("sum dict_df_X = {}".format(s))

print("\n-------------------------\n")

s = 0
for i in range(1, 6):
    s += dict_s_y[i].shape[0]
    print("dict_s_y | fold number {} | num rows: {}".format(i, dict_s_y[i].shape[0]))
print("sum dict_s_y = {}".format(s))

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [ ]:
def getAccuracy(s_y_hat, s_y_truth):
    return np.count_nonzero(s_y_truth.to_numpy()==s_y_hat.to_numpy()) / np.size(s_y_hat.to_numpy())


ones = pd.Series(np.ones(s_y.shape[0]))
getAccuracy(ones, s_y)

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` to search for the best hyperparameter. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [ ]:
# 5 outer folds
k = 5

# get the folded data
dict_df_X, dict_s_y = getDataFolds(df_X, s_y, k)

# store strings to print
crnt_imp_str = "Testing {} min impurity decrease"
avg_acc_str = "\tAverage accuracy over 4 folds is {}"
best_str = "\nBest min impurity decrease is {}\n"
    
    
# Q2.5, outer-fold accuracies for test and train sets.
test_acc = []
train_acc = []

# for every fold (k)
for i in range(1, k+1):

    # create 4 trees
    trees = []
    hp_MID = np.array([0.1,0.25,0.3,0.4])
    for hyper_param in hp_MID:
        trees.append(tree.DecisionTreeClassifier(min_impurity_decrease=hyper_param, criterion = "gini"))
    
    # get the test set
    test_X_df = dict_df_X[i]
    test_y_df = dict_s_y[i]

    # create the train set (4 folds)
    train_X_df = pd.DataFrame()
    train_y_df = pd.DataFrame()
    for j in range(1, k+1):
        # skip the test set
        if (j != i):
            train_X_df = train_X_df.append(dict_df_X[j])
            train_y_df = train_y_df.append(dict_s_y[j])

    
    accuracies = []
    # train each tree
    for t in trees:
        
        print(crnt_imp_str.format(t.min_impurity_decrease))
        
        # train tree
        t.fit(train_X_df, train_y_df)
        
        # predict each tree
        t_yhat = t.predict(test_X_df.to_numpy())
    
        # calculate accuracy
        acc = getAccuracy(pd.Series(t_yhat), test_y_df.squeeze())
        accuracies.append(acc)
        
        print(avg_acc_str.format(round(acc, 2)))
    
    # get highest accuracy tree
    h_idx = accuracies.index(max(accuracies))
    print(best_str.format(hp_MID[h_idx]))
        
        

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test and train sets.

In [ ]:
# based on the previous experiment, a min_impurity_decrease of 0.1 was 
# determined to be the best hyperparamter. 
# The min, max, mean accuracies accross 5-folds is shown.

In [ ]:
# 5 outer folds
k = 5

# get the folded data
dict_df_X, dict_s_y = getDataFolds(df_X, s_y, k)

# Create tree using best hyper parameter
t = tree.DecisionTreeClassifier(min_impurity_decrease=0.1, criterion = "gini")
    

train_accuracies = []
# for every fold (k)
for i in range(1, k+1):
    
    # get the test set
    test_X_df = dict_df_X[i]
    test_y_df = dict_s_y[i]

    # create the train set (4 folds)
    train_X_df = pd.DataFrame()
    train_y_df = pd.DataFrame()
    for j in range(1, k+1):
        # skip the test set
        if (j != i):
            train_X_df = train_X_df.append(dict_df_X[j])
            train_y_df = train_y_df.append(dict_s_y[j])


    # train tree
    t.fit(train_X_df, train_y_df)

    # predict the tree
    t_yhat = t.predict(test_X_df.to_numpy())

    # calculate accuracy
    acc = getAccuracy(pd.Series(t_yhat), test_y_df.squeeze())
    train_accuracies.append(acc)

print("--Generalized Performance--")
print("Min : {}\nMax : {}\nMean: {}".format(min(train_accuracies), max(train_accuracies), sum(train_accuracies)/len(train_accuracies)))